In [2]:
import pandas as pd
import numpy as np
import os
from utils2.Date_Processor import *
from utils2.Data_run import arg_set, model_run
from datetime import datetime, timedelta


### 데이터를 1년, 2년, 3년 단위 및 예측기간(3월13~3월19)을 6시간 단위로 쪼갠 기간의 직전 3년 데이터로 나눔

In [6]:
temp_list_1nd = range(2012,2023,1)
temp_list_2nd = [2012,2014,2016,2018,2020,2021]  # 마지막이 2022~2023되는 문제가 있어서 마지막만 2021로 바꿈
temp_list_3nd = [2012,2015,2018,2020,2021]  # 마지막이 2021~2023되는 문제가 있어서, 2020을 추가함.

In [ ]:
# 1년 단위 데이터 생성

for i in temp_list_3nd:
    year_interval = 3
    start_year = i
    end_year = start_year+year_interval
    
    date_filter = DateFilter(result_df, date_column='date')
    filtered_df = date_filter.filter_by_date(f'{start_year}-06-01', f'{end_year}-06-01')
    filtered_df.reset_index(inplace=True, drop=True)
    filtered_df = filtered_df.iloc[:-1]
    filtered_df.to_csv(f'./Data_Final(실제 모델 입력데이터)/3_year/{start_year}_{end_year}_{year_interval}_result.csv', index=False)

In [ ]:
result_df = pd.read_csv('./Data_Preprocessed(전처리 데이터)/Final_Preprocessed_Data.csv')
start_time = '2020-03-13 00:00:00'
end_time = '2023-03-12 23:55:00'

move_time = timedelta(hours=6)
date_format = '%Y-%m-%d %H:%M:%S'
start_datetime = datetime.strptime(start_time, date_format)
end_datetime = datetime.strptime(end_time, date_format)

for i in range(28):
    start = str(start_datetime)
    end = str(end_datetime)
    start_name = start.replace(' ', '_').replace(':', '-')
    end_name = end.replace(' ', '_').replace(':', '-')
    print(start, end)
    
    date_filter = DateFilter(result_df, date_column='date')
    filtered_df = date_filter.filter_by_date(start, end)
    filtered_df.reset_index(inplace=True, drop=True)
    filtered_df.to_csv("./Data_Final(실제 모델 입력데이터)/predict_period/{}__{}_result.csv" .format(start_name, end_name), index=False)
    
    start_datetime = start_datetime + move_time
    end_datetime = end_datetime + move_time

### 예측 데이터를 바탕으로 실제 예측치오차(MAPE를 산출함)

In [27]:
total_df = pd.read_csv('./Data_Preprocessed(전처리 데이터)/Final_Preprocessed_Data.csv')
data_folder_list = os.listdir('./results/')

In [28]:
colab_list = [element for element in data_folder_list if 'colab' in element]
local_list = [element for element in data_folder_list if 'colab' not in element]

In [29]:
local_list_1 = [element for element in local_list if '_1_' in element]
local_list_2 = [element for element in local_list if '_2_' in element]
local_list_3 = [element for element in local_list if '_3_' in element]
colab_list_1 = [element for element in colab_list if '_1_' in element]
colab_list_2 = [element for element in colab_list if '_2_' in element]
colab_list_3 = [element for element in colab_list if '_3_' in element]

In [24]:
def MAPE_search(folder_list):
    for i in folder_list:
        name = i.split('.csv')[0]
        data_path = './results/' + i + '/real_prediction.csv'
        temp_df = pd.read_csv(data_path)
        mean_mape = error_cal(total_df, temp_df, method='mape')
        
        print(f'File_Name : {name}, Mean_MAPE : {mean_mape}')
    

In [32]:
MAPE_search(local_list_2)

File_Name : D_Linear_2012_2014_2_result, Mean_MAPE : 0.03918087126044353
File_Name : D_Linear_2014_2016_2_result, Mean_MAPE : 0.030707298895060037
File_Name : D_Linear_2016_2018_2_result, Mean_MAPE : 0.025074300376841344
File_Name : D_Linear_2018_2020_2_result, Mean_MAPE : 0.025084640516588538
File_Name : D_Linear_2020_2022_2_result, Mean_MAPE : 0.046750485961244086
File_Name : D_Linear_2021_2023_2_result, Mean_MAPE : 0.007381055621748681
